<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/optimal_control_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optimal Control Overview

This page introduce various optimal control algorithms for solving a pendulum swing-up task. We will show computing and using:

1.   a pid controller;
2.   a lqr controller;
3.   an optimal numerical controller computed with value-iteration;
4.   an optimal trajectory computed with the direct collocation method;
5.   a trajectory following controller using the optimal trajectory.

**Importing Librairies**

This page uses the toolbox *pyro*.

In [ ]:
!git clone -b dev-alex https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')

In [ ]:
import numpy as np
from IPython import display
!apt install ffmpeg

**Defining the dynamical**

Here we load a already defined class from the library including the dynamic equations of the pendulum behavior, which is a function of the form:

$\dot{x} = f(x,u)$

where $x$ is the state vector $[\theta,\dot\theta]$, and $u$ is the torque. The class also inlcude other variables defining the control problem that we can set:
*   The max/min torque available;
*   The nominal state, set to $[\theta=-\pi,\dot\theta=0]$, which will be used as the goal state and the linearization point;
*   The intial state, set to $[\theta=0,\dot\theta=0]$, which will be used as a starting point for the simulations.


 

In [ ]:
##############################################################################
# Dynamics
##############################################################################

from pyro.dynamic  import pendulum

# Define the dynamical system to control
sys  = pendulum.SinglePendulum()

sys.u_ub[0] = +5 # Max torque
sys.u_lb[0] = -5 # Min torque

sys.xbar  = np.array([ -3.14 , 0 ]) # target state [ position , velocity ]
sys.x0    = np.array([ -0.00 , 0 ]) # initial state

The dynamical equations $\dot{x} = f(x,u)$ can be represented graphically by a vector field shown here:

In [ ]:
sys.plot_phase_plane() # Graphical illustration of the dynamic behavior in the phase plane

**Defining the cost function**

Most optimal control algorithm are based on optimising a "cost-function" which is a mathematical representation of the performance

Here we will use a standard quadratic cost function or the type:

$J = \int  ( x^T Q x + u^T R u ) dt$

In [ ]:
##############################################################################
# Cost Function
##############################################################################

from pyro.analysis import costfunction

cf = costfunction.QuadraticCostFunction.from_sys( sys ) 

cf.INF  = 10000     # The value iteration algo needs this parameter

cf.Q[0,0] = 1
cf.Q[1,1] = 0.1
cf.R[0,0] = 1

sys.cost_function = cf

Here we see the final matrix used to define the cost function:

In [ ]:
print('Q=\n',cf.Q)
print('R=\n',cf.R)

**PID controller**

Here as a baseline lets evaluate a basic PID controller. For now lets only use proportionnal gain $Kp$ and derivative gain $Kd$.

In [ ]:
##############################################################################
# Sub-optimal PID
##############################################################################

from pyro.control  import linear

pid      = linear.ProportionalController(1,2) # 1 output from 2 inputs
pid.rbar = sys.xbar                           # target
pid.K    = np.array([30,30])                   # gain matrix

print('PID: [ Kp , Kd ] = ',pid.K)

Here we illustrates the resulting control law which is a function with two inputs (pendulum actual position and velocity) and one output (the motor torque) of the form $τ=c (θ,\dotθ)$

In [ ]:
pid.plot_control_law( sys = sys )

Here we use the library tools to compute the equation of motion of the pendulum in closed-loop with the controller and to simulate a trajectory.

In [ ]:
sys_with_pid = pid + sys
sys_with_pid.plot_trajectory('xuj')

In [ ]:
video = sys_with_pid.generate_simulation_html_video()
html  = display.HTML(video)
display.display(html)

In terms of our performance criteria we observe that the final cost of the simulated trajectory with this controller is:

In [ ]:
print('Trajectory cost: ', sys_with_pid.traj.J[-1])

**LQR controller**

For the specific case of an unconstrained linear system and a quadratic cost function, there is an analytical solution that we call LQR. Here the system is non-linear but we can linearize our system to generate a linear approximation of the form:

$\dot x = f(x,u) \approx A x + B u $

here we use a numerical approach to compute the Jacobians of the fonction that correspond to the A and B matrix:

$A = \frac{ \partial f}{ \partial  x}$

$B = \frac{ \partial f}{ \partial  u}$

In [ ]:
##############################################################################
# LQR
##############################################################################

from pyro.dynamic  import statespace

# Linear model
ss  = statespace.linearize( sys )

print('A=\n',ss.A)
print('B=\n',ss.B)

Then using the linear approximation, we use the analytical solution of the optimal controller for the linear system and the quadratic cost function defined earlier. This is done by solving the analytical riccati equation. The resulting optimal controller is a linear control law of the form:

$u = - K x $

In [ ]:
from pyro.control import lqr

lqr_ctl = lqr.synthesize_lqr_controller( ss , cf , sys.xbar)

print('LQR K=\n',lqr_ctl.K)

Then here we illustrates the LQR controller control law.

In [ ]:
lqr_ctl.plot_control_law( sys = sys )

Here we simulate a trajectory with the LQR controller.

In [ ]:
sys_with_lqr = lqr_ctl + sys
sys_with_lqr.plot_trajectory('xuj')

In [ ]:
video = sys_with_lqr.generate_simulation_html_video()
html  = display.HTML(video)
display.display(html)

In [ ]:
print('Trajectory cost: ', sys_with_lqr.traj.J[-1])

**Value Iteration**

One algorithm can optimize directly a system with any type of non-linearity and also inlcuding any constraint. For low-dof systems we can discretize the state-space and solve the bellman equation on this grid using the algorithm called value-iteration. First we iterate on to compute $J^*(x)$ the optimal cost-to-go that correspond to the futur cost of the trajectory if optimal actions are taken. The iteration consists in computing the following update until convergence:

$J^*(x) ⇐ min_u [g(x,u) + J^*(x_{next})] \quad \forall x$

the with $J^*(x)$ we can then compute the optimal control law with:

$c^*(x) = argmin_u [g(x,u) + J^*(x_{next})] \quad \forall x$

In [ ]:
##############################################################################
# VI
##############################################################################

from pyro.planning import valueiteration
from pyro.planning import discretizer

# Value iteration algo
vi = valueiteration.ValueIteration_2D( discretizer.GridDynamicSystem( sys ) , cf )

vi.initialize()
vi.compute_steps(200) # To compute from sratch instead of loading the solution
#vi.load_data('/content/pyro/examples/demo/simple_pendulum_vi') # To load a pre-computed solution
vi.assign_interpol_controller()
vi_ctl = vi.ctl

The following figure show the computed cost-to-go.

In [ ]:
vi.plot_cost2go(200)

The results control law is a non-linear map shown here:

In [ ]:
vi_ctl.plot_control_law( sys = sys , n=1000)

Here we simulate a trajectory with the optimal controller generated with value iteration.

In [ ]:
sys_with_vi = vi_ctl + sys
sys_with_vi.plot_trajectory('xuj')

In [ ]:
video = sys_with_vi.generate_simulation_html_video()
html  = display.HTML(video)
display.display(html)

In [ ]:
print('Trajectory cost: ', sys_with_vi.traj.J[-1])

**Trajectory Optimisation**

Another approach is instead of trying to find the optimal controller directly, it is much faster to find an optimal trajectory from just an initial state. One approach is to formulate the problem as an constrained optimisation problem of the form:

$min \int_{t_0}^{t_f}  ( x^T Q x + u^T R u ) dt  $

subject to $\dot{x} = f(x,u)$

In [ ]:
##############################################################################
# Direct Collocation Trajectory Optimisation
##############################################################################

from pyro.planning import trajectoryoptimisation

planner = trajectoryoptimisation.DirectCollocationTrajectoryOptimisation( sys ,
                                                                          dt = 0.1,
                                                                          grid = 50)

planner.x_start = sys.x0
planner.x_goal  = sys.xbar

planner.compute_optimal_trajectory()


Here we show the resulting optimised trajectory.

In [ ]:
planner.show_solution()

**Trajectory following controller**

Finding an optimal trajectory is however only part of the solution. Simply executing the computed torque as a function of time would quickly lead to diverging from the computed optimal trajectory, the trajectory needs to be stabilized.

Here we will use a control method call "computed torque" to stabilise the trajectory.

In [ ]:
##############################################################################
# Stabilizing the optimal trajectory
##############################################################################

from pyro.control  import nonlinear

traj_ctl  = nonlinear.ComputedTorqueController( sys , planner.traj )  

traj_ctl.w0   = 2
traj_ctl.zeta = 1
traj_ctl.rbar = sys.xbar[0:1]

The resulting control law is a function of the state of also the time

In [ ]:
traj_ctl.plot_control_law( t = 0 , sys = sys , n=100)

In [ ]:
traj_ctl.plot_control_law( t = 5 , sys = sys , n=100)

Here we simulate a trajectory with the trajectory following controller:

In [ ]:
sys_with_traj_ctl = traj_ctl + sys
sys_with_traj_ctl.plot_trajectory('xuj')

In [ ]:
video = sys_with_traj_ctl.generate_simulation_html_video()
html  = display.HTML(video)
display.display(html)

In [ ]:
print('Trajectory cost: ', sys_with_traj_ctl.traj.J[-1])

We can see that using the scheme of first optimising a trajectory and then stabilizing it using a trajectory following controller led to a solution almost as good as the resulting solution found using the value-iteration algorithm but using much lest computations.